This is a continuation of the project to process covid data and see whether we can learn more about what causes covid deaths. The dataset for covid was from our world in data. It has been cleaned, explored, and most of the preprocessing has been done including scaling it.

We shall now import the data into the notebook:

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('../../Unit16FeatureEngineering/Capstone2Preprocessing/ScaledData.csv')

In [3]:
df.head()

,date,new_cases,new_deaths,new_cases_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_hosp_admissions,...,people_fully_vaccinated,total_boosters,new_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,stringency_index,tests_units_not performed,tests_units_tests performed
0,2020-01-22,-0.684809,-1.230281,-0.684809,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.9409,-0.497097,-3.781973,4.460654,-4.460654
1,2020-01-23,-0.684809,-1.230281,-0.684809,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.9409,-0.497097,-3.781973,4.460654,-4.460654
2,2020-01-24,-0.684802,-1.230281,-0.684802,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.9409,-0.497097,-3.781973,4.460654,-4.460654
3,2020-01-25,-0.684809,-1.230281,-0.684809,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.9409,-0.497097,-3.781973,4.460654,-4.460654
4,2020-01-26,-0.684788,-1.230281,-0.684788,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.9409,-0.497097,-3.781973,4.460654,-4.460654


In [4]:
df['date'] = pd.to_datetime(df['date'])

In [5]:
df.set_index('date', inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 815 entries, 2020-01-22 to 2022-04-15
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   new_cases                            815 non-null    float64
 1   new_deaths                           815 non-null    float64
 2   new_cases_per_million                815 non-null    float64
 3   reproduction_rate                    815 non-null    float64
 4   icu_patients                         815 non-null    float64
 5   icu_patients_per_million             815 non-null    float64
 6   hosp_patients                        815 non-null    float64
 7   hosp_patients_per_million            815 non-null    float64
 8   weekly_hosp_admissions               815 non-null    float64
 9   weekly_hosp_admissions_per_million   815 non-null    float64
 10  total_tests                          815 non-null    float64
 11  new_tests    

In [9]:
X = df.drop(['new_deaths'], axis=1)
y = df[['new_deaths']]

In [10]:
X

,new_cases,new_cases_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,...,people_fully_vaccinated,total_boosters,new_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,stringency_index,tests_units_not performed,tests_units_tests performed
date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,-0.684809,-0.684809,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,-1.121894,-1.254081,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.940900,-0.497097,-3.781973,4.460654,-4.460654
2020-01-23,-0.684809,-0.684809,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,-1.121894,-1.254081,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.940900,-0.497097,-3.781973,4.460654,-4.460654
2020-01-24,-0.684802,-0.684802,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,-1.121894,-1.254081,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.940900,-0.497097,-3.781973,4.460654,-4.460654
2020-01-25,-0.684809,-0.684809,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,-1.121894,-1.254081,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.940900,-0.497097,-3.781973,4.460654,-4.460654
2020-01-26,-0.684788,-0.684788,-2.064156,-1.252235,-1.252234,-1.139472,-1.139472,-1.121894,-1.121894,-1.254081,...,-0.940899,-0.500739,-0.758638,-0.936484,-0.983961,-0.940900,-0.497097,-3.781973,4.460654,-4.460654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-11,-0.383126,-0.383126,0.797451,-1.072014,-1.072011,-0.881618,-0.881618,-0.835729,-0.835724,-1.254081,...,1.446885,2.554187,-0.247425,1.699904,1.459048,1.446802,2.581803,0.024797,-0.224182,0.224182
2022-04-12,-0.491969,-0.491969,0.797451,-1.076960,-1.076950,-0.881900,-0.881899,-0.832645,-0.832645,-1.254081,...,1.447260,2.556284,-0.413497,1.701446,1.459365,1.447165,2.583866,0.024797,-0.224182,0.224182
2022-04-13,-0.399617,-0.399618,0.797451,-1.077305,-1.077294,-0.881234,-0.881234,-0.830517,-0.830519,-1.254081,...,1.447315,2.556612,-0.726404,1.701446,1.459365,1.447165,2.583866,0.024797,-0.224182,0.224182


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=40)

In [12]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)

In [13]:
print(mean_squared_error(y_test, y_pred))
(r2_score(y_test, y_pred))

0.3765299802356845


0.6522232636314598

In [14]:
lm.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': False,
 'positive': False}

In [15]:
lr = LinearRegression()
grid_search = GridSearchCV(lr, param_grid={'copy_X': [True,False], 'fit_intercept': [True, False], 'n_jobs': [-1], 'normalize': [False, True], 'positive': [False, True]}, cv=5)

In [16]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False], 'n_jobs': [-1],
                         'normalize': [False, True],
                         'positive': [False, True]})

In [17]:
y_pred2 = grid_search.predict(X_test)

In [18]:
print(mean_squared_error(y_test, y_pred2))
(r2_score(y_test, y_pred2))

0.3754878693165293


0.653185795045747

The r2 score of the model is at 65.3%. This is not a very high accuracy, but it is better than the mean.

As mentioned in the EDA notebook, the most telling features from this dataset are total_tests_per_thousand, people_fully_vaccinated, tests_units_not performed, positive_rate, reproduction_rate, new_vaccinations.